# Working with AGOL items with the ArcGIS Python API
Now that we've covered how to connect to AGOL and find specific datasets, we'll now examine how we access these data as various programming objects and what we can do with those object. 

We'll start where the last notebooks left of: working with the [Census Tract data](db3f9c8728dd44e4ad455e0c27a85eea) (item ID = `db3f9c8728dd44e4ad455e0c27a85eea`. 

#### Resources:
* https://developers.arcgis.com/python/guide/working-with-feature-layers-and-features/

### Step 1. Connecting to AGOL and fetching the data item

In [ ]:
#Import the GIS object and connect (this time anonymously)
from arcgis import GIS
gis = GIS()

In [ ]:
#Fetch the census tract data from its item ID
tractsItem = gis.content.get('20f5d275113e4066bf311236d9dcc3d4')

In [ ]:
#Display the item
tractsItem

In [ ]:
#Reveal the data type
type(tractsItem)

In [ ]:
#Reveal the the type of Item
tractsItem.type

---
►  ***Knowledge check*** What is the item type for the item with the ID `427c34604a76466ba001fbc80817082a`

In [ ]:
#Show the type of item 427c34604a76466ba001fbc80817082a


### Step 2. Exploring the item returned
The result we obtained is a generic AGOL "Item" ([Reference](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#item)). The item we retrieved happens to represent a **Feature Service** item (as seen above), and as such it has defined properties and methods that we can access programatically. 

To see what these are, we can retrieve the API reference on the Feature Layer object:<br>
https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#featurelayer

What's important to understand here is the nestedness of objects served on AGOL. And this can be seen by accessing our object's REST endpoint:

In [ ]:
#Show the URL of the feature server's REST endpoint
print(tractsItem.url)

Click on the above link to access its REST endpoint. 

There you see indeed that it is a Feature Server. You also can see what **Feature Layers** are contained in this feature service:

>**Layers:**
>* [USA Census Tract Boundaries ](https://services.arcgis.com/P3ePLMYs2RVChkJx/ArcGIS/rest/services/USA_Census_Tracts/FeatureServer/0) (0)

This indicates that we have just one feature layer associated with the feature service. We can reveal this via the ArcGIS Python API via the Feature Layers `layers` property:

In [ ]:
#Extract the layer(s) included with this feature service
tractLayers = tractsItem.layers
tractLayers

This returns a list with just one layer in it -- just as we found in the browser based exploration. Now we extract that layer into its own coding object and reveal the data type of this object. 

In [ ]:
#Pull the one layer item associated with the service to a new variable
tractsLayer = tractLayers[0]
type(tractsLayer)

We see this object is something called a **"FeatureLayer"**. We can explore the help on this object to see just what we can do with this kind of thing. Note also, however, the *FeatureLayer* object is a subset of the *Layer* object, and thus some of the operations of the Layer object will apply to the FeatureLayer object as well. So, we'd want to investigate the documentation on that object too. (In code speak, this is called *inheritance*: FeatureLayers inherit properies and methods from the Layer object...)

→ More info on the ArcGIS `layer` object: https://developers.arcgis.com/python/latest/api-reference/arcgis.gis.toc.html#layer<br>
→ More info on the ArcGIS `FeatureLayer` object: https://developers.arcgis.com/python/latest/api-reference/arcgis.features.toc.html#featurelayer

#### Mapping
The ArcGIS Python API has some nice mapping capability. Detailed information is at the link below, but a quick example goes a long way.
* https://developers.arcgis.com/python/guide/using-the-map-widget/

In [ ]:
#Map the feature layer
m = gis.map('Durham, NC') #Create the map object, centered on Durham, NC
m.add_layer(tractsLayer)  #Adds the census tracts layer to the map
m                         #Displays the map

In [ ]:
#Map just Durham records and color by a field value
map2 = gis.map('Durham County, NC') #Create the map object, centered on Durham, NC
map2.add_layer(tractsLayer,{"definition_expression":"STCOFIPS = '37063'", #Filter records for just Durham 
                            "renderer":"ClassedColorRenderer",            #Create a color renderer object
                            "field_name":"POPULATION_2020",               #Color based on the POPULATION field values
                            "opacity":0.7                                 #Lighten the opacity
                         })  
map2

---

### Step 5. Explore properties of the FeatureLayer object
The [`properties`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#arcgis.features.FeatureLayer.properties) function of the FeatureLayers object returns a dictionary of properties. 
* Print the entire `properties` object to expose all the properties...

In [ ]:
#Reveal all properties of the feature layer
tractsLayer.properties

* Print specific properties. What is the `service item ID` of our feature layer? Its `name`? Its `capabilities`? 

In [ ]:
#Report the tractsLayer "itemID", "name", and "capabilities"
print("Service item ID:", tractsLayer.properties.serviceItemId)
print("Name:",tractsLayer.properties.name)
print("Capabilities",tractsLayer.properties.capabilities)

* The `fields` property returns another dictionary. Save that to a new variable then iterate through each field and print the field's name

In [ ]:
#Iterate through all fields and report the field's name
for fld in tractsLayer.properties.fields:
    print (fld.name, end=',')

In [ ]:
#Same as above, but using "list comprehension"
print([fld.name for fld in tractsLayer.properties.fields])

### Step 6. Interact with our Feature Layer
Moving beyond just the properties, we can apply some of the methods associated with the feature layer. Most methods are for updating data, which we can't do, but we can list unique values and also subset records via a query. 

* List all the **unique values** in the STATE column in the feature layer

In [ ]:
#List the unique values found in the STATE attribute. 
print(tractsLayer.get_unique_values('STATE_ABBR'))

* **Query** the records in the Feature Layer

In [ ]:
#Subset records that are in Durham Co (FIPS 37063)
fset_Durham = tractsLayer.query(where="STCOFIPS LIKE '37063%'")
type(fset_Durham)

The object returned is a **Feature Set**. ([Reference on feature set](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#featureset))

In [ ]:
#Display the number of results returned
len(fset_Durham)

* Create a quick map of the feature set.<br>
→ _Note feature sets use the map object's `draw()` function, whereas the feature layer above uses the `.add_layer()` function._ <br>
→ _Also note that sometimes you need to run the process twice to show the data..._

In [ ]:
#Map the feature set with 'draw'
map3 = gis.map('Durham County, NC')
map3.basemap = 'osm' #(run map3.basemaps to create a list of basemap options)
map3.draw(fset_Durham,popup={'title':'FIPS','content':'POP20_SQMI'})
map3

* Report the geometry type of the FeatureSet

In [ ]:
#What geometry type is the result
fset_Durham.geometry_type

* Convert the featureset to a spatial dataframe with its `sdf` command

In [ ]:
sdf_Durham = fset_Durham.sdf
sdf_Durham.head()

### Step 7. Save our data to a local file!
* The FeatureSet has another function called `save` which allows us to download our queried results.

In [ ]:
#Save the selected features to a shapefile
fset_Durham.save(save_location='.',out_name='DurhamTracts')

---
### Success!

We just downloaded our own copy of the Durham County Tracts file. We can now grab any feature layer we've found on ArcGIS Online. Well, it's not always this easy as datasets to have some download limits:

In [ ]:
tractsLayer.properties.maxRecordCount

We can get around that restriction by "paging" our download, i.e. downloading in chunks of records, 2000 at a time...

For more info on this process, see ESRI's documentation on querying feature layers:<br>
https://developers.arcgis.com/python/guide/working-with-feature-layers-and-features/#Querying-feature-layers

### Step 8. Analyzing the data here, as a dataframe
Of course, why stop there. We have our data in our coding environment. Let's analyze it!

To facilitate analyses, we can convert our featureset to (1) a list of [Feature](https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html) objects or (2) to a [spatial dataframe](https://developers.arcgis.com/python/guide/introduction-to-the-spatially-enabled-dataframe/). 

In [ ]:
#Get all the tracts for North Carolina
nc_tracts = tractsLayer.query("STATE_ABBR = 'NC'")
len(nc_tracts)

In [ ]:
#Convert feature set to a list of features
features = nc_tracts.features
len(features)

In [ ]:
#Grab the first feature
feature= features[0]
type(feature)

In [ ]:
feature.get_value('FIPS')

In [ ]:
#Convert the feature set data as a dataframe (takes a moment...)
sdf = nc_tracts.sdf
sdf.head()

* Note the output has a column called "SHAPE". These values are ArcGIS API `geometry` objects. 

#### Analyzing geometry of a single feature

In [ ]:
#Extract the first row as a feature
theFeature = sdf.loc[0]
theFeature

In [ ]:
#Get the feature's "SHAPE" value
theGeom = theFeature['SHAPE']
type(theGeom)

In [ ]:
#Get the feature's spatial reference (google the WKID to learn more)
theGeom.spatial_reference

In [ ]:
#Get the feature's area, in the spatial reference's units
theGeom.area

https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.geometry.html#arcgis.geometry.Geometry.get_area

In [ ]:
#Get the area, in square miles, measured geodesically
theGeom.get_area(method='GEODESIC',units='MILES')

#### Analyzing demographics by county

In [ ]:
#Group the data by values in the COUNTY field
county_sum = sdf.groupby('STCOFIPS').sum(numeric_only=True)

In [ ]:
#Show the first 5 records in the result
county_sum['POPULATION'].head()

In [ ]:
#Summarize the county data
county_sum['POPULATION'].describe()

In [ ]:
%matplotlib inline
county_sum['POPULATION'].sort_values().plot(kind='barh',figsize=(10,20));

## Summary
The learning curve steepens a bit on figuring out how to work with the data we can retrieve using the ArcGIS Python API. However, with patience, perserverence, and a structured understanding of both how the data are served and of the documentation, you can use the ArcGIS Python API to do a tremendous amount of useful coding. 

We've focused on data access, analysis, and visualization, but if you browse the notebook examples that ESRI provides, you'll see that the level of work you can use with this library is quite astounding. Furthermore, as GIS (and all computing really) moves from the desktop to the cloud, these types of analysis are likely to dominate the research landscape before too long. 